In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Oct  8 02:29:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
import pathlib
data = pathlib.Path('/content/gdrive/MyDrive/Data')

In [ ]:
!pip install qiskit==0.32.1

In [ ]:
!git clone https://github.com/mit-han-lab/torchquantum.git

In [ ]:
%cd torchquantum

In [ ]:
!pip install --editable .

In [ ]:
%env PYTHONPATH=.

In [ ]:
from qiskit import IBMQ
IBMQ.save_account('9a5832193632e80ba59c14b34726764801700a548a68bd023fc2631656e4723f5fd248bf2f9b5cbb766ee37ca76aea3c55cbbeb71117beb65452aec92b5e3338', overwrite=True)


In [ ]:
!pip install matplotlib==3.1.3

In [ ]:
!ls artifact

In [ ]:
!cp artifact/aerbackend.py ../../usr/local/lib/python3.7/dist-packages/qiskit/providers/aer/backends/ -r

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchquantum as tq
import random
from torch.optim import lr_scheduler
import time
import os
import copy
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
torch.manual_seed(42)
np.random.seed(42)
import matplotlib.pyplot as plt
os.environ["OMP_NUM_THREADS"] = "1"
import torch.nn as nn
import torch.nn.functional as F 
import torchquantum as tq
import torchquantum.functional as tqf

In [ ]:
batch_size=10
step = 0.0001               
num_epochs =6             
gamma_lr_scheduler = 0.1    
q_delta = 0.01              
start_time = time.time()

In [ ]:
data_transforms = {
    'train': transforms.Compose([
       
        transforms.Resize((64,64)),
        transforms.RandomHorizontalFlip(),
        
        transforms.ToTensor(),
        transforms.Normalize([0.3854, 0.3854, 0.3854], [0.4143, 0.4143, 0.4143]),
        
        
      
      transforms.Grayscale(num_output_channels=1),
        
        
    ]),
    
    'test': transforms.Compose([
        
         transforms.Resize((64,64)),
        
       
        transforms.ToTensor(),
        transforms.Normalize([0.3854, 0.3854, 0.3854], [0.4143, 0.4143, 0.4143]),
        
        
        transforms.Grayscale(num_output_channels=1),
        
    ]),
     'val': transforms.Compose([
        
         transforms.Resize((64,64)),
        
       
        transforms.ToTensor(),
        transforms.Normalize([0.3854, 0.3854, 0.3854], [0.4143, 0.4143, 0.4143]),
        
        
        transforms.Grayscale(num_output_channels=1),
        
    ]),
}

data_dir = data
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train','test','val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10,
                                             shuffle=True, num_workers=2)
              for x in ['train','test','val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','test','val']}
class_names = image_datasets['train'].classes

In [ ]:
class QuanvolutionFilter(tq.QuantumModule):
    def __init__(self):
        super().__init__()
        self.n_wires = 4
        self.q_device = tq.QuantumDevice(n_wires=self.n_wires)
        self.encoder = tq.GeneralEncoder(
        [   {'input_idx': [0], 'func': 'ry', 'wires': [0]},
            {'input_idx': [1], 'func': 'ry', 'wires': [1]},
            {'input_idx': [2], 'func': 'ry', 'wires': [2]},
            {'input_idx': [3], 'func': 'ry', 'wires': [3]},
          ])
      

        self.q_layer = tq.RandomLayer(n_ops=8, wires=list(range(self.n_wires)))
        self.measure = tq.MeasureAll(tq.PauliZ)
        
    @tq.static_support
    def forward(self, x, use_qiskit=False):
        bsz = x.shape[0]
        size = 64
        x = x.view(bsz, size, size)
        
       
        data_list = []

        for c in range(0, size, 2):
            for r in range(0, size, 2):
                data = torch.transpose(torch.cat((x[:, c, r], x[:, c, r+1], x[:, c+1, r], x[:, c+1, r+1])).view(4, bsz), 0, 1)
                if use_qiskit:
                    data = self.qiskit_processor.process_parameterized(
                        self.q_device, self.encoder, self.q_layer, self.measure, data)
                else:
                    self.encoder(self.q_device, data)
                    self.q_layer(self.q_device)
                    data = self.measure(self.q_device)

                data_list.append(data.view(bsz, 4))
        
        result = torch.cat(data_list, dim=1).float()
        
        return result

In [ ]:
class HybridModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qf = QuanvolutionFilter()
        #self.conv1=nn.Conv2d(in_channels=1, out_channels=4, kernel_size=2,stride=2)
        
        self.linear = torch.nn.Linear(4*32*32,3)
    
    def forward( self, x, use_qiskit=False):
        x = x.reshape(-1,1,64,64)
        #x = self.conv1(x).to(device)
        with torch.no_grad():
          x = self.qf(x,use_qiskit)
        x=F.relu(x).to(device)
        x = x.reshape(-1,4*32*32)
        x = self.linear(x)
        return F.log_softmax(x, -1)

        

        
       
      
                              
       

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model_hybrid = HybridModel().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
optimizer_hybrid = optim.Adam(model_hybrid.parameters(), lr=0.0001)

In [ ]:
exp_lr_scheduler = lr_scheduler.StepLR(
    optimizer_hybrid, step_size=10, gamma=gamma_lr_scheduler
)

In [ ]:
tloss=[]
vloss=[]
tacc=[]
vacc=[]

def train_model(model, criterion, optimizer, scheduler, num_epochs=15):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs,use_qiskit=False).to(device)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                tloss.append(epoch_loss)
                tacc.append(epoch_acc)
            if phase == 'test':
                vloss.append(epoch_loss)
                vacc.append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_hybrid = train_model(
    model_hybrid, criterion, optimizer_hybrid, exp_lr_scheduler, num_epochs=15
)

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)

nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,use_qiskit=False)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;

In [ ]:
total_params = sum(p.numel() for p in model_hybrid.parameters())
total_params

In [ ]:
learn_params = sum(p.numel() for p in model_hybrid.parameters() if p.requires_grad)
learn_params

In [ ]:
tacc1=[]
for i in range (15):
    tacc1.append(tacc[i].cpu())

In [ ]:
vacc1=[]
for i in range (15):
    vacc1.append(vacc[i].cpu())

In [ ]:

plt.plot(tacc1,'-o')
plt.plot(vacc1,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')

plt.show()
plt.savefig("acc1.png")

In [ ]:
plt.plot(tloss,'-o')
plt.plot(vloss,'-o')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Loss')

plt.show()
plt.savefig("loss1.png")

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,use_qiskit=False)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;
figure = heatmap.get_figure()    
figure.savefig('heatmaptest.png', dpi=400)

In [ ]:
from qiskit import IBMQ
from torchquantum.plugins import QiskitProcessor
backend_name = 'ibmq_jakarta'
print(f"\nTest on Real Quantum Computer {backend_name}")
processor_real_qc = QiskitProcessor(use_real_qc=True,backend_name=backend_name,hub='ibm-q-research-2', group='tribhuvan-uni-1', project='main')
model_hybrid.qf.set_qiskit_processor(processor_real_qc)

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,use_qiskit=True)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;

In [ ]:
from qiskit import IBMQ
from torchquantum.plugins import QiskitProcessor
backend_name = 'ibm_lagos'
print(f"\nTest on Real Quantum Computer {backend_name}")
processor_real_qc = QiskitProcessor(use_real_qc=True,backend_name=backend_name,hub='ibm-q-research-2', group='tribhuvan-uni-1', project='main')
model_hybrid.qf.set_qiskit_processor(processor_real_qc)

In [ ]:
testloader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)
correct, total = 0, 0
correct_count, all_count = 0, 0
for images,labels in testloader:
  for i in range(len(labels)):
    if torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
        
    img = images[i].view(-1,1, 28, 28)
    with torch.no_grad():
        logps = model_hybrid(images,use_qiskit=True)

    
    ps = torch.exp(logps)
    probab = list(ps.cpu()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.cpu()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

In [ ]:
from qiskit import IBMQ
from torchquantum.plugins import QiskitProcessor
backend_name = 'ibmq_perth'
print(f"\nTest on Real Quantum Computer {backend_name}")
processor_real_qc = QiskitProcessor(use_real_qc=True,backend_name=backend_name,hub='ibm-q-research-2', group='tribhuvan-uni-1', project='main')
model_hybrid.qf.set_qiskit_processor(processor_real_qc)

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,use_qiskit=True)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;

In [ ]:
from qiskit import IBMQ
from torchquantum.plugins import QiskitProcessor
print(f"\nTest with Qiskit Simulator")
processor_simulation = QiskitProcessor(use_real_qc=False)
model_hybrid.qf.set_qiskit_processor(processor_simulation)

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,True)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;

In [ ]:
from qiskit import IBMQ
from torchquantum.plugins import QiskitProcessor
print(f"\nTest with Qiskit Simulator")
processor_simulation = QiskitProcessor(use_real_qc=False,backend_name='simulator_statevector')
model_hybrid.qf.set_qiskit_processor(processor_simulation)

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,True)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;

In [ ]:
from qiskit import IBMQ
from torchquantum.plugins import QiskitProcessor
print(f"\nTest with Qiskit Simulator")
processor_simulation = QiskitProcessor(use_real_qc=False,backend_name='simulator_extended_stabilizer')
model_hybrid.qf.set_qiskit_processor(processor_simulation)

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,True)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;

In [ ]:
from qiskit import IBMQ
from torchquantum.plugins import QiskitProcessor
print(f"\nTest with Qiskit Simulator")
processor_simulation = QiskitProcessor(use_real_qc=False,backend_name='simulator_mps')
model_hybrid.qf.set_qiskit_processor(processor_simulation)

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,True)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;

In [ ]:
from qiskit import IBMQ
from torchquantum.plugins import QiskitProcessor
print(f"\nTest with Qiskit Simulator")
processor_simulation = QiskitProcessor(use_real_qc=False,backend_name='simulator_stabilizers')
model_hybrid.qf.set_qiskit_processor(processor_simulation)

In [ ]:
import seaborn as sns
import pandas as pd
test_loader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=10, shuffle=True, num_workers=2)
nb_classes = 3
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
       
        outputs = model_hybrid(inputs,True)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

plt.figure(figsize=(5,5))

#class_names = list(label2class.values())
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=5)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=5)
plt.ylabel('True label')
plt.xlabel('Predicted label')
;